In [ ]:
!pip install trl peft datasets

In [ ]:

import torch
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, set_seed
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split = "train")

In [ ]:
model_name = 'microsoft/Phi-3-mini-4k-instruct'

compute_dtype = torch.bfloat16
attn_implementation = 'flash_attention_2'

lora_r = 8
lora_alpha = 16
lora_dropout = 0.05

target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
set_seed(42)  # For reproducibility

In [ ]:
def apply_peft_chat_template(examples):
    formatted_text = (
        f"<|instruction|> {examples['instruction']} <|end|>\n"
        f"<|input|> {examples['input']} <|end|>\n"
        f"<|output|> {examples['output']} <|end|>\n"
    )
    return {'formatted_text': formatted_text}

formatted_dataset = dataset.map(apply_peft_chat_template, batched=False)

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=compute_dtype, trust_remote_code=True)
# load tokerinzer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Add special tokens if needed
special_tokens = {'additional_special_tokens': ['<|instruction|>', '<|input|>', '<|output|>', '<|end|>']}
tokenizer.add_special_tokens(special_tokens)

tokenizer.padding_side = 'right' # to prevent warnings

# Tokenize the formatted dataset
def tokenize_function(examples):
    return tokenizer(examples['formatted_text'], truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=['instruction', 'input', 'output', 'formatted_text'])

train_test_split = tokenized_dataset.train_test_split(test_size=0.5)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
model.gradient_checkpointing_enable()

peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= target_modules
)

# Define the training arguments
args = TrainingArguments(
        output_dir="./phi-3-mini-custom",
        evaluation_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_strategy="epoch",
        logging_steps=100,
        learning_rate=1e-4,
        eval_steps=100,
        num_train_epochs=3,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
        seed=42,
)

trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
        max_seq_length=512,
        tokenizer=tokenizer,
        args=args,
)